In [1]:
import pandas as pd
import numpy as np
test_set = pd.read_csv("testing_set.csv")
feature_matrix = pd.read_csv("feature_matrix.csv")


In [2]:
#get user_id from testing set
temp = test_set.columns.values[1:]
test_user_id = []
for i in range(len(temp)):
    test_user_id.append(temp[i])
for i in range(len(test_user_id)):
    test_user_id[i] = int(test_user_id[i])
print("ID of users in the testing set is:")
print(test_user_id)

ID of users in the testing set is:
[37015, 37020, 37032, 37034, 37040, 37049, 37052, 37074, 37097, 37100, 37107, 37110, 37117, 37133, 37149, 37163, 37196, 37199, 37219, 37229, 37233, 37236, 37237, 37264, 37267, 37270, 37326, 37327, 37345, 37372, 37383, 37386, 37399, 37405, 37411, 37423, 37425, 37454, 37459, 37464, 37490, 37494, 37502, 37508, 37520, 37524, 37549, 37558, 37576, 37599, 37609, 37621, 37641, 37662, 37674, 37698, 37702, 37720, 37725, 37754, 37781, 37800, 37803, 37806, 37808, 37823, 37873, 37874, 37903, 37916, 37922, 37928, 37965, 37971, 37979, 37985, 37998, 38004, 38009, 38042, 38045, 38079, 38083, 38087, 38100, 38120, 38137, 38152, 38162, 38213, 38233, 38237, 38253, 38263, 38275, 38278, 38290, 38312, 38318, 38322, 38324, 38367, 38381, 38383, 38388, 38433, 38436, 38462, 38467, 38471, 38487, 38494, 38515, 38521, 38525, 38531, 38537, 38541, 38544, 38555, 38557, 38582, 38586, 38599, 38622, 38644, 38732, 38747, 38753, 38757, 38769, 38777, 38790, 38797, 38803, 38813, 38820, 38833

In [3]:

def get_SubX_and_SubY(n:int):
    #user n
    column_n = str(n) 
    sub_y = test_set[test_set[column_n]!=0][['anime_id',column_n]]
    merge_table = pd.merge(feature_matrix,sub_y,on =  'anime_id') 
    subY = merge_table[column_n].values
    subX = merge_table['genre_vector'].values
    subX = np.array([eval(vector) for vector in subX])
    del sub_y,merge_table
    total,N = 0,0
    for rating in subY:
        if rating != -1:
            total += rating
            N+=1
    if N!=0:
        for i in range(len(subY)):
            if subY[i] == -1:
                subY[i] = round(total/N,2)
    else:
        for i in range(len(subY)):
            subY[i] = 5
    row,col = subX.shape
    #since subX and subY has the same number of row
    #70% and 30% remaining of subX
    subX1,subX2 = subX[:(row*7)//10],subX[-(row*3)//10::]
    #70% and 30% remaining of subY
    subY1,subY2 = subY[:(row*7)//10],subY[-(row*3)//10::]
    del subX,total,N,subY
    return subX1,subX2,subY1,subY2

_,__,___,____ = get_SubX_and_SubY(37020)
print(_,__,___,____)


[[1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]] [[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [10.    8.   10.   10.    7.   10.   10.   10.   10.   10.   10.   10.
 10.    8.    6.    7.    8.43 10.   10.   10.   10.    8.   10.    9.
  8.   10.    8.    4.   10.    9.    7.   10.   10.   10.   10.    7.
  6.    8.    8.   10.   10.   10.    7.   10.    6.   10.   10.   10.
 10.    6.    6.    9.    7.    8.43  7.   10.   10.    8.   10.    8.
  9.    7.    8.    7.    6.    8.   10.   10.    6.   10.    7.    8.
  6.    9.    8.    7.    7.    8.   10.   10.    6.    8.    9.    8.
 10.    9.    7.    8.    8.   10.    8.   10.    8.    9.    9.    9.
  7.   10.   10.  ] [ 7.   10.    7.    7.    8.43  9.    8.   10.    9.   10.    9.   10.
  9.    9.    7.    9.    8.    7.    5.    5.    8.43  8.    8.    7.
 10.    7.   10.    8.    8.  

In [4]:
import Ridge
solver = Ridge.RidgeRegression(0.01 , 10000 , 50)

In [5]:
#evaluation function
def evaluate(user_id:int):
    train_featureVector,test_featureVector,y_train,y_true = get_SubX_and_SubY(user_id)
    weight,bias = solver.fit(train_featureVector,y_train)
    y_predicted = solver.predict(test_featureVector)
    
    
    return np.mean((y_true-y_predicted)**2)


In [7]:
#calculate the RMSE of all user in the testing set
n = len(test_user_id)
num = 0
total_rmse = 0
for test_user in test_user_id:
    if num > 150:
        break
    total_rmse += evaluate(test_user)
    num += 1


solver.RidgeInfo()
print("RMSE of testing set with {} users is:".format(num))
print(total_rmse/num)

Learning rate:  0.01
Iterations:  10000
Lambda:  50
RMSE of testing set with 151 users is:
1.4420292319176722
